In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
import pathlib
import os

from helper_functions import *
metadata = pd.read_csv("metadata.csv")

import warnings
warnings.filterwarnings("ignore")

import re
pattern = re.compile("^[1-9][0-9]M")

# for topK in [500, 1000]:
#    for atlas_sample_types in ["Tissue", "Tissue,WBC", "Tissue,WBC,Control", "Tissue,Control"]:
for topK in [500]:
    for atlas_sample_types in ["Tissue,WBC"]:
        datadir = "./datadir"
        # outputdir = "./outputdir_02102023"
        outputdir = "outputdir_23102023_hypo_hyper"
        path_to_03_output = os.path.join(outputdir, "03_output_noFDR")
        path_to_04_output = os.path.join(outputdir, "04_output_noFDR", "top{}_{}".format(topK, atlas_sample_types.replace(",", "_and_")))
        os.system("mkdir -p {}".format(path_to_04_output))
        
        for sampletype in ["cfDNA", "Tissue", "WBC", "Control"]:
        # for sampletype in ["Control"]:
            selected_metadata = metadata[metadata["Sample type"].isin([sampletype])]
            
            outputfile = os.path.join(path_to_04_output,  
                                           "top{}_atlas_{}.all_{}_betadf.csv".format(topK, atlas_sample_types.replace(",", "_and_"), sampletype))
            
            if os.path.exists(outputfile) == False:
                atlas = pd.read_csv(os.path.join(path_to_03_output, "top{}_atlas_{}.final.csv".format(topK, atlas_sample_types)), index_col =[0])
                
                atlas = atlas[[item for item in atlas.columns if "_y" not in item ]]
                atlas.columns = [item.replace("_x", "") for item in atlas.columns]
                
                avg_atlas = atlas.set_index("sample").fillna(0).groupby("label").mean()
                atlas_regions = [item for item in atlas.columns if item not in ["sample", "label"]]
                
                all_betadf = pd.DataFrame(data = [item for item in selected_metadata["filename"].to_list()], columns = ["sample"])
                
                for region in tqdm(atlas_regions):
                    if region not in all_betadf.columns:
                        file = "./{}/read_at_region_{}.csv".format(readdf_dir[sampletype], region)
                        df = pd.read_csv(file, index_col = [0])
                        
                        if df.shape[0] != 0:
                            region_chrom = region.split("_")[0]
                            region_start = int(region.split("_")[1])
                            region_end = int(region.split("_")[2])
                            refseq_at_cluster = get_refseq(path_to_all_fa = path_to_all_fa, 
                                                                chrom = region_chrom, 
                                                                start = region_start, 
                                                                end = region_end + 1)
                            all_cpg_in_cluster = [m.start(0) for m in re.finditer("CG", refseq_at_cluster)]
                            cpg_coords = [item + region_start for item in all_cpg_in_cluster]
                            
                            df["label"] = df["sample"].apply(lambda x: metadata[metadata["filename"] == x]["Label"].unique()[0])
                            
                            df["check_cigar"] = df["cigar"].apply(lambda x: bool(pattern.fullmatch(x)))
                            df = df[df["check_cigar"] == True]
                            
                            df["Label"] = df["sample"].apply(lambda x: metadata[metadata["filename"] == x]["Label"].unique()[0])
                            
                            if df.shape[0] != 0:
                                df["end"] = df[["start", "cigar"]].apply(lambda x: int(x[0]) + int(x[1].replace("M", "")), axis = 1)
                                for cpg_pos in cpg_coords:
                                    df[cpg_pos] = df[["start", "end", "seq"]].apply(lambda x: get_CpG_status(x[0], x[1], x[2], cpg_pos, mode = "num"), axis = 1)
                                
                                betadf = selected_metadata[["filename"]].copy()
                                betadf.columns = ["sample"]
                                
                                for cpg_pos in cpg_coords:    
                                    tmpdf = df[["sample", cpg_pos, "Label"]].copy()
                                    tmpcountdf = tmpdf.groupby('sample')[cpg_pos].apply(lambda x: (x == 1).sum()/((x == 0).sum() + (x == 1).sum()) ).reset_index(name= "meth_level_{}".format(cpg_pos))
                                    betadf = betadf.merge(tmpcountdf[["sample", "meth_level_{}".format(cpg_pos)]], right_on = "sample", left_on = "sample", how = "outer")
                                
                                betadf["avg_beta"] = betadf[[item for item in betadf.columns if item != "sample"]].apply(lambda x: np.mean([item for item in x if np.isnan(item) == False]), axis = 1)
                                betadf["label"] = betadf["sample"].apply(lambda x: metadata[metadata["filename"] == x]["Label"].unique()[0])
                                betadf = betadf[["sample", "avg_beta"]]
                                betadf.columns = ["sample", region]
                                all_betadf = all_betadf.merge(betadf[["sample", region]], right_on = "sample", left_on = "sample")
                
                all_betadf.to_csv(os.path.join(path_to_04_output,  
                                               "top{}_atlas_{}.all_{}_betadf.csv".format(topK, atlas_sample_types.replace(",", "_and_"), sampletype)))

100%|███████████████████████████████████████| 5943/5943 [25:09<00:00,  3.94it/s]
